# Dataset-to-Hub pipeline: reduce Time-to-Publish in onboarding datasets 

Goal: from a PI System with associated AF database, publish the dataset through OCS with a set of default asset-centric Data Views. 


**----- Changelog from v2: new schema data -----**

#### Showcase datasets: Deschutes and UC Davis Facilities. Next: NC State paper machines, USC drill data

## Step 1: Populate GraphQL-enabled "AF" 

#### From an AF path element, collect pipoints/static/etc about all its children elements. 

#### This data is then directly accessible through a GraphQL endpoint for next steps


In [1]:
# import to deal with PIWebAPI
# !pip install httpx
import requests
import json
import time
from time import process_time
import yaml
import asyncio
import httpx
import os

from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport
import urllib3

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# Graph module
import py2neo
from py2neo import Node, Relationship

In [2]:
config_file = "config-delmar.yaml"
# config_file = "config-cmu2.yaml"
# config_file = "config-omf-health.yaml"
# config_file = "config-prod-ucd-v2.yaml"
# config_file = "config-windfarm.yaml"
# config_file = "config-acad-prod-desc-v2.yaml"
# config_file = "config-acad-prod-deschutes.yaml"
#  config_file = "config-acad-prod-ucd.yaml"

## Overview

From an anchor element path in AF, populate a GraphQL-enabled "mirror" of its children elements and their attributes, extracted from PIWebAPI 

### Input parameters

* PIWebAPI base URL for asset servers
* PIWebAPI authentication credentials (basic)
* Anchor element path

In [3]:
with open(config_file) as f:
    config = yaml.safe_load(f)
# config

In [4]:
base_url = config["piwebapi"]["base_url"]
dataserver = config["piwebapi"]["dataserver"]  # "uni-pida-vm0"

auth = (config["piwebapi"]["username"], config["piwebapi"]["password"])
timeout = 45.0

asset_url = base_url + "/assetservers"

print(f"piwebapi={base_url}, dataserver={dataserver}, auth=(*****, *****)")

element_roots = config["piwebapi"]["element_roots"]

test_asset = None

ocs_asset_db = config["ocs"]["configuration"]["asset_db"]

EXPAND_LEAF_ELEMENTS = config["piwebapi"]["expand_leaf_elements"]  # False
MAX_LEAF_ELEMENTS = 10000
print(f"roots = {element_roots}, expand leaves: {EXPAND_LEAF_ELEMENTS}")

piwebapi=https://academicpi.osisoft.com/piwebapi, dataserver=acad-pida-vm0, auth=(*****, *****)
roots = ['\\\\PIAF-ACAD\\PilotPlant\\Plant\\Pilot Plant'], expand leaves: True


### Functions to navigate PIWebAPI structure 

In [5]:
def map_webid(webid):
    return f"{ocs_asset_db}:{webid}"


# From PIWebAPI asset node, build directory of children links and self WebID
# If extract_key is present, return this directory item
# Otherwise return full dictionary (key is children node name)
async def extract_url(url, key, extract_key=None, debug=False):
    async with httpx.AsyncClient() as client:
        start = process_time()
        print("<", end="")
        r = await client.get(url, auth=auth, timeout=timeout)
        # print(f"[{process_time() - start:.2f}:{url}]")
        if r.status_code != 200:
            print(f"@error  code={r.status_code}, url={url}")
            return
        js = r.json()
        if debug:
            print(f"js={js}")
        d = {
            i["Name"]: (
                i["Links"][f"{key}"],
                map_webid(i["WebId"]),
                i.get("TemplateName", ""),
                i.get("Description", ""),
            )
            for i in js["Items"]
        }
        if extract_key:
            result = d.get(extract_key, None)
            if result is None:
                print(
                    f"[@error url={url}, key={key}, extract={extract_key}, d={d}]"
                )  # " r={js}, d={d}")
            return result
        else:
            return d


# Extract static value v2
async def extract_static_value2(attr_info, client):
    return await extract_point_or_value(
        attr_info, client, "Value", "Value", lambda t: t
    )


# Extract point data v2
async def extract_point2(attr_info, client):
    # "Path", lambda t: extract_tag(t)
    return await extract_point_or_value(
        attr_info, client, "Point", "Name", lambda t: f"tag__{t}"
    )


# Extract point or static value from attribute
async def extract_point_or_value(attr_info, client, link_key, value_key, value_f):
    if attr_info["Links"].get(link_key, None) is None:
        return None, None, None, None
    start = process_time()
    print(".", end="")
    r = await client.get(attr_info["Links"][link_key], auth=auth, timeout=timeout)
    # print(f"[{process_time() - start:.2f}:{attr_info['Links'][link_key]}]")
    if r.status_code != 200:
        print(f"@get error code={r.status_code} url={attr_info['Links'][link_key]}")
        return
    js = r.json()
    point_attributes = None
    if link_key == "Point":
        r2 = await client.get(js["Links"]["Attributes"], auth=auth, timeout=timeout)
        if r2.status_code != 200:
            print(f"@get error code={r.status_code} url2={js['Links']['Attributes']}")
            return
        point_attributes = r2.json()["Items"]
    return attr_info, value_f(js[value_key]), js, point_attributes


def create_or_update_node(
    node_type, name, webid, template="", asset_db="", parent=None
):
    ##query = f"MATCH (node) WHERE node.id = '{webid}' RETURN node"

    nodes = []  ## [i["node"] for i in graph.run(query).data()]
    if len(nodes) == 1:
        print("@", end="")
        return nodes[0]
    node = Node(node_type, name=name)  ### node_type, "Node"
    node.update(id=webid, af_template=template, asset_db=asset_db)
    # if node_type == "PIPoint":
    #    node.update(on_ocs=1)
    return node


important_nodes = {}

# Navigate fully AF `path` in PIWebAPI
# If `graph` is not null, update it while traversing tree
# (graph at this point is a list of Node and Relationship)
async def elements_of(url, path, lgraph=None):
    path_components = enumerate([i for i in path.split("\\") if len(i) > 0])
    # url = base_url
    parent = None
    asset_db = ""
    for i, component in path_components:
        # url, webid, template, description
        result = await extract_url(
            url, "Databases" if i == 0 else "Elements", component
        )
        if result is None:
            print(f"[@@@error: url={url}, i={i}, component={component}]")
            continue
        url, webid, template, description = result
        if lgraph is not None:
            node_type = "Server" if i == 0 else ("Database" if i == 1 else "Element")
            if node_type == "Database":
                important_nodes[node_type] = webid

            rel_type = (
                "NOT_POSSIBLE"
                if i == 0
                else ("HAS_DATABASE" if i == 1 else "HAS_ELEMENT")
            )
            if node_type == "Database":
                asset_db = ocs_asset_db  # component
            node = create_or_update_node(
                node_type, component, webid, template, asset_db
            )
            if node_type == "Database":
                node.update(
                    name=config["db"]["database_name"],
                    informationURL=config["db"]["infoURL"],
                    description=config["db"]["description"],
                    namespace=config["ocs"]["configuration"]["namespace"],
                    status="onboarding",
                )
                important_nodes[node_type] = node
            elif node_type == "Element":
                node.update(description=description if description != "" else template)
            if parent:
                lgraph.append(Relationship(parent, rel_type, node))
            lgraph.append(node)
            parent = node

    if important_nodes.get("Element", None) is None:
        important_nodes["Element"] = node
    print(f"important nodes={important_nodes}")
    start = process_time()
    urls_dict = await extract_url(url, "Attributes")
    # print(f"[{process_time() - start:.2f}:extract_url_{url}]")
    return urls_dict, parent


# From an element URL, return a dictionary of attribute and their PIWebAPI JSON info
# PIWebAPI attribute JSON to be parsed and transfered to graph
async def attributes(url):
    async with httpx.AsyncClient() as client:
        start = process_time()
        print(">", end="")
        r = await client.get(url, auth=auth, timeout=timeout)
        # print(f"[{process_time() - start:.2f}:{url}]")
        if r.status_code != 200:
            print("@error")
            return
        js = r.json()
        d = {
            i["Name"]: i for i in js["Items"] if i["DataReferencePlugIn"] == "PI Point"
        }
        for k in d.keys():  # key_list
            # print(f"d-key={k}")
            d[k] = await extract_point2(d[k], client)

        d2 = {i["Name"]: i for i in js["Items"] if i["DataReferencePlugIn"] == ""}
        for k in d2.keys():
            # print(f"d2-key={k}")
            d2[k] = await extract_static_value2(d2[k], client)

        return {**d, **d2}


def convert_config_data(value):
    if type(value) == float:
        if str(value)[-2:] == ".0":
            return int(value)
        else:
            return value
    try:
        return int(value)
    except (ValueError, TypeError):
        try:
            val = value["Name"]
            return val.replace("'", "")  # f"'{val}'"
        except:
            return value.replace("'", "")  # "'{value}'"

### Transform linear graph (list of Node and Relationship) into list of GraphQL mutations

In [6]:
# GraphQL client object
sample_transport = RequestsHTTPTransport(
    url=config["graphql"]["endpoint"], verify=False, retries=3
)
client = Client(transport=sample_transport, fetch_schema_from_transport=True)

In [7]:
def commit_graph(g, debug=True):
    start_time = time.perf_counter()
    # tx = graph.begin()
    if debug:
        print(f"\n>> Graph root={g[0]}")
    # [tx.create(i) for i in g if i is not None]
    for node in g:
        if node is None:
            print("### [None]")
        else:
            if type(node) == py2neo.data.Node:
                ntype = str(node.labels).replace(":", "").replace("Node", "")
                d = dict(node)
                d.pop("step", None)  ## temporary
                d.pop("on_ocs", None)  ## temporary
                pre_args = ",".join([f' {key}: "{d[key]}"' for key in d.keys()])
                args = pre_args.replace('"True"', "true").replace('"False"', "false")
                print("@@@@@", ntype, d)
                node_mutation = gql(
                    f"""
                mutation 
                   {{ Merge{ntype}({args}) {{
                       id 
                       name
                   }}
                   }}
                """
                )
                client.execute(node_mutation)
    for relation in g:
        if relation is None:
            print("### [None]")
        else:
            if type(relation) != py2neo.data.Node:  # must a relationship
                start_node = relation.start_node
                end_node = relation.end_node
                ntype = str(start_node.labels).replace(":", "").replace("Node", "")
                merge_suffix = type(relation).__name__.capitalize()
                args = f"""from: {{id: \"{start_node['id']}\"}}, to: {{id: \"{end_node['id']}\"}}"""
                print("&&&", ntype, args)
                mutation = f""" 
                mutation {{ 
                    Merge{ntype}{merge_suffix}({args}) {{
                        from {{
                            name
                        }}
                        to {{
                            name
                        }}
                    }}
                }}
                """
                relation_mutation = gql(mutation)
                ## print(">>>>> mutation=", mutation)
                reply = client.execute(relation_mutation)
                print("<<<<<<", reply)
    # tx.commit()
    if debug:
        print(f">> commit done in {time.perf_counter() - start_time:.2f} secs")

### Create all children element nodes of anchor element and their attributes

For attributes, transfer relevent information onto associated graph node

In [8]:
def reject_element(element_name):
    return any(ss in element_name.lower() for ss in ["_cache", "baseline", "health"])

try:
    exec(config["reject_element"])
except:
    pass

In [9]:
async def generate_leaf_element_attributes(elements_path):
    lgraph = []
    elem_attr_urls, elem_anchor = await elements_of(
        asset_url, elements_path, lgraph=lgraph
    )
    print(f"\n>> Current anchor: {elem_anchor}")
    # elem_anchor, len(lgraph if lgraph else []), list(elem_attr_urls.keys())
    for element_name in elem_attr_urls.keys():
        # UC Davis + CMU specific
        if reject_element(element_name):
            print(f"@@ rejected: {element_name}")
            continue
        name = element_name
        # print(elem_attr_urls[element_name])
        elem_url, webid, template, description = elem_attr_urls[element_name]
        elem_node = create_or_update_node(
            "Element", name, webid, template, asset_db=elem_anchor["asset_db"]
        )
        elem_node.update(description=description if description != "" else template)
        elem_rel = Relationship(elem_anchor, "HAS_ELEMENT", elem_node)
        attributes_info = await attributes(elem_attr_urls[element_name][0])
        static_attributes = []
        for attr in attributes_info.keys():
            try:
                attr_info, stream_or_val, stream_js, point_attrs = attributes_info[attr]
            except ValueError:
                print(f"attr={attr}  val={attributes_info[attr]}")
                return
            if attr_info is None:
                continue
            config_attribute = "tag__" not in str(stream_or_val)
            if not config_attribute:
                stream_or_val = stream_or_val.replace("tag__", "")
            node = create_or_update_node(
                "Attribute",
                attr,
                map_webid(attr_info["WebId"]),
                template=elem_node["af_template"],
                asset_db=elem_node["asset_db"],
            )
            node.update(
                value=convert_config_data(stream_or_val),
                type=attr_info["Type"],
                is_stream=not config_attribute,
                virtual=False,
                parent=element_name,
                # configString=fr"{attr_info['ConfigString']}",
            )
            rel = Relationship(elem_node, "HAS_ATTRIBUTE", node)
            lgraph.extend([node, rel])
            if config_attribute:
                static_attributes += [attr]
                continue
            else:  # execute rest of code since it's a pipoint
                attribute_node = node
                web_id = os.path.split(attr_info["Links"]["Point"])[1]
                stream_name = stream_or_val

            if "Analysis" in attr:  # NOTE: duplicate tag in attributes for FV
                # skip attribute with duplicate tag
                print("=", end="")
                continue
            if stream_js["Future"]:
                # future tag not transferred by PItoOCS
                print(f"${attr}$", end="")
                continue
            node = create_or_update_node(
                "PIPoint",
                attr,
                web_id,  # map_webid(attr_info["WebId"]),
                elem_node["af_template"],
                elem_node["asset_db"],
            )
            print("+", end="")  # stream_name
            pattributes = {i["Name"]: i["Value"] for i in point_attrs}
            node.update(
                asset_id=name,
                column_name=attr,
                stream_name=stream_name,
                type=attr_info["Type"],
                uom=attr_info["DefaultUnitsName"],
                step=attr_info["Step"],
                categories=attr_info["CategoryNames"],
                description=attr_info["Description"],
                pointsource=pattributes["pointsource"],
                stream_id=f"PI_{config['piwebapi']['dataserver']}_{pattributes['pointid']}",
            )
            if (
                attr_info["Type"] == "EnumerationValue"
                or stream_js["PointType"] == "Digital"
            ):
                # print(f"node={node}, js={stream_js}")
                node.update(digital_set_name=stream_js["DigitalSetName"])
            rel = Relationship(attribute_node, "HAS_DYNAMIC", node)
            lgraph.extend([node, rel])
        # elem_node.update(static_attributes=static_attributes)
        lgraph.extend([elem_node, elem_rel])
    commit_graph(lgraph)

### Generate and commit graphs (one per anchor elements)

In [10]:
async def build_graphs(roots):
    for elements_path in roots:
        # lgraph = []
        await generate_leaf_element_attributes(elements_path)  # , lgraph)

In [11]:
print(element_roots)
start_time = time.perf_counter()
await build_graphs(element_roots)
print(f"> runtime {time.perf_counter() - start_time:.2f} secs")

['\\\\PIAF-ACAD\\PilotPlant\\Plant\\Pilot Plant']
<<<<important nodes={'Database': Node('Database', af_template='', asset_db='pilot.plant', description='Petroleum Industry and Process Technology Pilot Plant', id='pilot.plant:F1RDcFWpYyj-GkWKwoVLgdSfpwG1XyxiSvX0WMNyDlsRHq8AUElBRi1BQ0FEXFBJTE9UUExBTlQ', informationURL='https://vikingnews.delmar.edu/osisoft-gifts-over-1-million-in-software-to-process-technology-program-and-enhances-training-for-industry-workforce', name='Pilot_Plant', namespace='hub_lab_data', status='onboarding'), 'Element': Node('Element', af_template='', asset_db='pilot.plant', description='', id='pilot.plant:F1EmcFWpYyj-GkWKwoVLgdSfpw7j7_VF8k6xGpdwANOjT1RQUElBRi1BQ0FEXFBJTE9UUExBTlRcUExBTlRcUElMT1QgUExBTlQ', name='Pilot Plant')}
<
>> Current anchor: (:Element {af_template: '', asset_db: 'pilot.plant', description: '', id: 'pilot.plant:F1EmcFWpYyj-GkWKwoVLgdSfpw7j7_VF8k6xGpdwANOjT1RQUElBRi1BQ0FEXFBJTE9UUExBTlRcUExBTlRcUElMT1QgUExBTlQ', name: 'Pilot Plant'})
>..........

@@@@@ Attribute {'name': 'Feed Flow Controller Output', 'id': 'pilot.plant:F1AbEcFWpYyj-GkWKwoVLgdSfpwBj__VF8k6xGpdwANOjT1RQWL6dsXpjvE2vVZwM9Otj-AUElBRi1BQ0FEXFBJTE9UUExBTlRcUExBTlRcUElMT1QgUExBTlRcSE9ULTF8RkVFRCBGTE9XIENPTlRST0xMRVIgT1VUUFVU', 'af_template': '', 'asset_db': 'pilot.plant', 'value': 'delmar.DELMAR_UNIT1_FIC-101.OUT', 'type': 'Single', 'is_stream': True, 'virtual': False, 'parent': 'HOT-1'}
@@@@@ PIPoint {'name': 'Feed Flow Controller Output', 'id': 'F1DPpjXwPKxU8EWdWQR_TqpF6Q3i8AAAQUNBRC1QSURBLVZNMFxERUxNQVIuREVMTUFSX1VOSVQxX0ZJQy0xMDEuT1VU', 'af_template': '', 'asset_db': 'pilot.plant', 'asset_id': 'HOT-1', 'column_name': 'Feed Flow Controller Output', 'stream_name': 'delmar.DELMAR_UNIT1_FIC-101.OUT', 'type': 'Single', 'uom': 'percent', 'categories': [], 'description': 'FIC-101.OUT', 'pointsource': 'PIC', 'stream_id': 'PI_acad-pida-vm0_12254'}
@@@@@ Attribute {'name': 'Feed Flow Controller PV', 'id': 'pilot.plant:F1AbEcFWpYyj-GkWKwoVLgdSfpwBj__VF8k6xGpdwANOjT1RQ4Moajdu

@@@@@ PIPoint {'name': 'H-101 Outlet Temperature Controller Setpoint', 'id': 'F1DPpjXwPKxU8EWdWQR_TqpF6QmC8AAAQUNBRC1QSURBLVZNMFxERUxNQVIuREVMTUFSX1VOSVQxX1RJQy0xMDQuU1A', 'af_template': '', 'asset_db': 'pilot.plant', 'asset_id': 'HOT-1', 'column_name': 'H-101 Outlet Temperature Controller Setpoint', 'stream_name': 'delmar.DELMAR_UNIT1_TIC-104.SP', 'type': 'Single', 'uom': 'degree Fahrenheit', 'categories': [], 'description': 'TIC-104.SP', 'pointsource': 'PIC', 'stream_id': 'PI_acad-pida-vm0_12184'}
@@@@@ Attribute {'name': 'P-101 A/B Discharge pressure', 'id': 'pilot.plant:F1AbEcFWpYyj-GkWKwoVLgdSfpwBj__VF8k6xGpdwANOjT1RQS0pOuOuGkE6AgsuIjjwd_QUElBRi1BQ0FEXFBJTE9UUExBTlRcUExBTlRcUElMT1QgUExBTlRcSE9ULTF8UC0xMDEgQS9CIERJU0NIQVJHRSBQUkVTU1VSRQ', 'af_template': '', 'asset_db': 'pilot.plant', 'value': 'delmar.DELMAR_UNIT1_PI-111.PV', 'type': 'Single', 'is_stream': True, 'virtual': False, 'parent': 'HOT-1'}
@@@@@ PIPoint {'name': 'P-101 A/B Discharge pressure', 'id': 'F1DPpjXwPKxU8EWdWQR_Tqp

@@@@@ Attribute {'name': 'Bottoms Temperature', 'id': 'pilot.plant:F1AbEcFWpYyj-GkWKwoVLgdSfpw8T7_VF8k6xGpdwANOjT1RQ4IEsqdLCIEaxV3FXtkfbfAUElBRi1BQ0FEXFBJTE9UUExBTlRcUExBTlRcUElMT1QgUExBTlRcSE9ULTN8Qk9UVE9NUyBURU1QRVJBVFVSRQ', 'af_template': '', 'asset_db': 'pilot.plant', 'value': 'delmar.DELMAR_UNIT3_TI-200.PV', 'type': 'Single', 'is_stream': True, 'virtual': False, 'parent': 'HOT-3'}
@@@@@ PIPoint {'name': 'Bottoms Temperature', 'id': 'F1DPpjXwPKxU8EWdWQR_TqpF6Qoy8AAAQUNBRC1QSURBLVZNMFxERUxNQVIuREVMTUFSX1VOSVQzX1RJLTIwMC5QVg', 'af_template': '', 'asset_db': 'pilot.plant', 'asset_id': 'HOT-3', 'column_name': 'Bottoms Temperature', 'stream_name': 'delmar.DELMAR_UNIT3_TI-200.PV', 'type': 'Single', 'uom': 'degree Fahrenheit', 'categories': ['Process Data'], 'description': 'TI-200.PV', 'pointsource': 'PIC', 'stream_id': 'PI_acad-pida-vm0_12195'}
@@@@@ Attribute {'name': 'Bottoms to Feed Flow Controller Mode', 'id': 'pilot.plant:F1AbEcFWpYyj-GkWKwoVLgdSfpw8T7_VF8k6xGpdwANOjT1RQqQvgU8AGCkG7

@@@@@ Attribute {'name': 'Overhead Receiver Level Controller Output', 'id': 'pilot.plant:F1AbEcFWpYyj-GkWKwoVLgdSfpw8T7_VF8k6xGpdwANOjT1RQ7DEH8sADS0atf4LwWJqNKwUElBRi1BQ0FEXFBJTE9UUExBTlRcUExBTlRcUElMT1QgUExBTlRcSE9ULTN8T1ZFUkhFQUQgUkVDRUlWRVIgTEVWRUwgQ09OVFJPTExFUiBPVVRQVVQ', 'af_template': '', 'asset_db': 'pilot.plant', 'value': 'delmar.DELMAR_UNIT3_LIC-240.OUT', 'type': 'Single', 'is_stream': True, 'virtual': False, 'parent': 'HOT-3'}
@@@@@ PIPoint {'name': 'Overhead Receiver Level Controller Output', 'id': 'F1DPpjXwPKxU8EWdWQR_TqpF6Q3y8AAAQUNBRC1QSURBLVZNMFxERUxNQVIuREVMTUFSX1VOSVQzX0xJQy0yNDAuT1VU', 'af_template': '', 'asset_db': 'pilot.plant', 'asset_id': 'HOT-3', 'column_name': 'Overhead Receiver Level Controller Output', 'stream_name': 'delmar.DELMAR_UNIT3_LIC-240.OUT', 'type': 'Single', 'uom': 'percent', 'categories': ['Process Data'], 'description': 'LIC-240.OUT', 'pointsource': 'PIC', 'stream_id': 'PI_acad-pida-vm0_12255'}
@@@@@ Attribute {'name': 'Overhead Receiver Level Co

...

<<<<<< {'MergeElementHas_attribute': {'from': {'name': 'HOT-1'}, 'to': {'name': 'H-101 Outlet Pressure'}}}
&&& Attribute from: {id: "pilot.plant:F1AbEcFWpYyj-GkWKwoVLgdSfpwBj__VF8k6xGpdwANOjT1RQSSQ5EObFpke8lUZyU_Wx9QUElBRi1BQ0FEXFBJTE9UUExBTlRcUExBTlRcUElMT1QgUExBTlRcSE9ULTF8SC0xMDEgT1VUTEVUIFBSRVNTVVJF"}, to: {id: "F1DPpjXwPKxU8EWdWQR_TqpF6Qky8AAAQUNBRC1QSURBLVZNMFxERUxNQVIuREVMTUFSX1VOSVQxX1BJLTEwOS5QVg"}
<<<<<< {'MergeAttributeHas_dynamic': {'from': {'name': 'H-101 Outlet Pressure'}, 'to': {'name': 'H-101 Outlet Pressure'}}}
&&& Element from: {id: "pilot.plant:F1EmcFWpYyj-GkWKwoVLgdSfpwBj__VF8k6xGpdwANOjT1RQUElBRi1BQ0FEXFBJTE9UUExBTlRcUExBTlRcUElMT1QgUExBTlRcSE9ULTE"}, to: {id: "pilot.plant:F1AbEcFWpYyj-GkWKwoVLgdSfpwBj__VF8k6xGpdwANOjT1RQxVm9Sj3WakWm-Vkk1gtJVgUElBRi1BQ0FEXFBJTE9UUExBTlRcUExBTlRcUElMT1QgUExBTlRcSE9ULTF8SC0xMDEgT1VUTEVUIFRFTVBFUkFUVVJFIENPTlRST0xMRVIgT1VUUFVU"}
<<<<<< {'MergeElementHas_attribute': {'from': {'name': 'HOT-1'}, 'to': {'name': 'H-101 Outlet Temperature C

<<<<<< {'MergeElementHas_attribute': {'from': {'name': 'HOT-1'}, 'to': {'name': 'Product Water Flow'}}}
&&& Attribute from: {id: "pilot.plant:F1AbEcFWpYyj-GkWKwoVLgdSfpwBj__VF8k6xGpdwANOjT1RQ5QjAFzrZtEKFEs4dEWm34AUElBRi1BQ0FEXFBJTE9UUExBTlRcUExBTlRcUElMT1QgUExBTlRcSE9ULTF8UFJPRFVDVCBXQVRFUiBGTE9X"}, to: {id: "F1DPpjXwPKxU8EWdWQR_TqpF6QyC8AAAQUNBRC1QSURBLVZNMFxERUxNQVIuREVMTUFSX1VOSVQxX0ZJLTEwMi5QVg"}
<<<<<< {'MergeAttributeHas_dynamic': {'from': {'name': 'Product Water Flow'}, 'to': {'name': 'Product Water Flow'}}}
&&& Element from: {id: "pilot.plant:F1EmcFWpYyj-GkWKwoVLgdSfpwBj__VF8k6xGpdwANOjT1RQUElBRi1BQ0FEXFBJTE9UUExBTlRcUExBTlRcUElMT1QgUExBTlRcSE9ULTE"}, to: {id: "pilot.plant:F1AbEcFWpYyj-GkWKwoVLgdSfpwBj__VF8k6xGpdwANOjT1RQ_4f6h9fQGkaSYRQ72hAtGgUElBRi1BQ0FEXFBJTE9UUExBTlRcUExBTlRcUElMT1QgUExBTlRcSE9ULTF8VU5JVCAxIENPTU1PTiBUUk9VQkxFIEFMQVJN"}
<<<<<< {'MergeElementHas_attribute': {'from': {'name': 'HOT-1'}, 'to': {'name': 'Unit 1 Common Trouble Alarm'}}}
&&& Attribute from: {id: "p

<<<<<< {'MergeElementHas_attribute': {'from': {'name': 'HOT-3'}, 'to': {'name': 'Cooling Water Header Temperature'}}}
&&& Attribute from: {id: "pilot.plant:F1AbEcFWpYyj-GkWKwoVLgdSfpw8T7_VF8k6xGpdwANOjT1RQRgONrR6dHUKrYPvNv56c3gUElBRi1BQ0FEXFBJTE9UUExBTlRcUExBTlRcUElMT1QgUExBTlRcSE9ULTN8Q09PTElORyBXQVRFUiBIRUFERVIgVEVNUEVSQVRVUkU"}, to: {id: "F1DPpjXwPKxU8EWdWQR_TqpF6QvC8AAAQUNBRC1QSURBLVZNMFxERUxNQVIuREVMTUFSX1VOSVQzX1RJLTI2MC5QVg"}
<<<<<< {'MergeAttributeHas_dynamic': {'from': {'name': 'Cooling Water Header Temperature'}, 'to': {'name': 'Cooling Water Header Temperature'}}}
&&& Element from: {id: "pilot.plant:F1EmcFWpYyj-GkWKwoVLgdSfpw8T7_VF8k6xGpdwANOjT1RQUElBRi1BQ0FEXFBJTE9UUExBTlRcUExBTlRcUElMT1QgUExBTlRcSE9ULTM"}, to: {id: "pilot.plant:F1AbEcFWpYyj-GkWKwoVLgdSfpw8T7_VF8k6xGpdwANOjT1RQP0XiXJ6-2UiPO9jBK4Lo1AUElBRi1BQ0FEXFBJTE9UUExBTlRcUExBTlRcUElMT1QgUExBTlRcSE9ULTN8RkVFRCBURU1QRVJBVFVSRQ"}
<<<<<< {'MergeElementHas_attribute': {'from': {'name': 'HOT-3'}, 'to': {'name': 'Feed Tempera

<<<<<< {'MergeAttributeHas_dynamic': {'from': {'name': 'Reboiler Outlet Temperature Controller Mode'}, 'to': {'name': 'Reboiler Outlet Temperature Controller Mode'}}}
&&& Element from: {id: "pilot.plant:F1EmcFWpYyj-GkWKwoVLgdSfpw8T7_VF8k6xGpdwANOjT1RQUElBRi1BQ0FEXFBJTE9UUExBTlRcUExBTlRcUElMT1QgUExBTlRcSE9ULTM"}, to: {id: "pilot.plant:F1AbEcFWpYyj-GkWKwoVLgdSfpw8T7_VF8k6xGpdwANOjT1RQTRi_JQW-oEqDOco7M0mHGAUElBRi1BQ0FEXFBJTE9UUExBTlRcUExBTlRcUElMT1QgUExBTlRcSE9ULTN8UkVCT0lMRVIgT1VUTEVUIFRFTVBFUkFUVVJFIENPTlRST0xMRVIgT1VUUFVU"}
<<<<<< {'MergeElementHas_attribute': {'from': {'name': 'HOT-3'}, 'to': {'name': 'Reboiler Outlet Temperature Controller Output'}}}
&&& Attribute from: {id: "pilot.plant:F1AbEcFWpYyj-GkWKwoVLgdSfpw8T7_VF8k6xGpdwANOjT1RQTRi_JQW-oEqDOco7M0mHGAUElBRi1BQ0FEXFBJTE9UUExBTlRcUExBTlRcUElMT1QgUExBTlRcSE9ULTN8UkVCT0lMRVIgT1VUTEVUIFRFTVBFUkFUVVJFIENPTlRST0xMRVIgT1VUUFVU"}, to: {id: "F1DPpjXwPKxU8EWdWQR_TqpF6Qwi8AAAQUNBRC1QSURBLVZNMFxERUxNQVIuREVMTUFSX1VOSVQzX1RJQy0yMDUuT1VU"}
<

<<<<<< {'MergeAttributeHas_dynamic': {'from': {'name': 'Tower Top Bed Differential Pressure'}, 'to': {'name': 'Tower Top Bed Differential Pressure'}}}
&&& Element from: {id: "pilot.plant:F1EmcFWpYyj-GkWKwoVLgdSfpw8T7_VF8k6xGpdwANOjT1RQUElBRi1BQ0FEXFBJTE9UUExBTlRcUExBTlRcUElMT1QgUExBTlRcSE9ULTM"}, to: {id: "pilot.plant:F1AbEcFWpYyj-GkWKwoVLgdSfpw8T7_VF8k6xGpdwANOjT1RQ_cG2qk5eyUSGZ7VXzjGnjwUElBRi1BQ0FEXFBJTE9UUExBTlRcUExBTlRcUElMT1QgUExBTlRcSE9ULTN8VE9XRVIgVE9QIFNFQ1RJT04gVEVNUEVSQVRVUkU"}
<<<<<< {'MergeElementHas_attribute': {'from': {'name': 'HOT-3'}, 'to': {'name': 'Tower Top Section Temperature'}}}
&&& Attribute from: {id: "pilot.plant:F1AbEcFWpYyj-GkWKwoVLgdSfpw8T7_VF8k6xGpdwANOjT1RQ_cG2qk5eyUSGZ7VXzjGnjwUElBRi1BQ0FEXFBJTE9UUExBTlRcUExBTlRcUElMT1QgUExBTlRcSE9ULTN8VE9XRVIgVE9QIFNFQ1RJT04gVEVNUEVSQVRVUkU"}, to: {id: "F1DPpjXwPKxU8EWdWQR_TqpF6Qyy8AAAQUNBRC1QSURBLVZNMFxERUxNQVIuREVMTUFSX1VOSVQzX1RJLTIyNS5QVg"}
<<<<<< {'MergeAttributeHas_dynamic': {'from': {'name': 'Tower Top Section Tem

In [12]:
db_node = important_nodes["Database"]
elem_root = important_nodes["Element"]
root_webid = elem_root["id"]
print(elem_root)

(:Element {af_template: '', asset_db: 'pilot.plant', description: '', id: 'pilot.plant:F1EmcFWpYyj-GkWKwoVLgdSfpw7j7_VF8k6xGpdwANOjT1RQUElBRi1BQ0FEXFBJTE9UUExBTlRcUExBTlRcUElMT1QgUExBTlQ', name: 'Pilot Plant'})


### (Optional) From a test element, produce the list of all attributes

Data Views are created from different subset of this list

In [13]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i : i + n]


async def gather_func(f, items, ndiv):
    div = (len(items) // ndiv) + 1
    chunk_list = list(chunks(items, div))
    coroutines = [f(chunk_list[i]) for i in range(0, ndiv)]
    start_time = time.perf_counter()
    print("-OK-") if await asyncio.gather(*coroutines) else print("@oops")
    print(f"> runtime {time.perf_counter() - start_time:.2f} secs")

In [14]:
# define reject_element function if needed in config yaml
try:
    exec(config["reject_leaf_element"])
except:
    pass

query_leaves = gql(
    """
query Database($asset_db: String) {
    Database(asset_db: $asset_db) {
        leaf_elements {
            name
        }
    }
}"""
)


if EXPAND_LEAF_ELEMENTS:
    ndiv = 3
    root = element_roots[0]
    # query = f"MATCH (e:Element) WHERE NOT ((e)-[:HAS_ELEMENT]->()) and e.asset_db='{ocs_asset_db}' RETURN DISTINCT e.name AS name ORDER BY e.name"
    # leaf_elements = [i["name"] for i in graph.run(query).data()]
    result = client.execute(query_leaves, variable_values={"asset_db": ocs_asset_db})
    print(result)
    leaf_elements = [i["name"] for i in result["Database"][0]["leaf_elements"]]
    roots = [root + f"\\{element}" for element in leaf_elements[:MAX_LEAF_ELEMENTS]]
    print(len(roots), roots)
    if len(roots) > 3:
        await gather_func(build_graphs, roots, ndiv)
    else:
        await build_graphs(roots)

{'Database': [{'leaf_elements': [{'name': 'HOT-1'}, {'name': 'HOT-3'}]}]}
2 ['\\\\PIAF-ACAD\\PilotPlant\\Plant\\Pilot Plant\\HOT-1', '\\\\PIAF-ACAD\\PilotPlant\\Plant\\Pilot Plant\\HOT-3']
<<<<<important nodes={'Database': Node('Database', af_template='', asset_db='pilot.plant', description='Petroleum Industry and Process Technology Pilot Plant', id='pilot.plant:F1RDcFWpYyj-GkWKwoVLgdSfpwG1XyxiSvX0WMNyDlsRHq8AUElBRi1BQ0FEXFBJTE9UUExBTlQ', informationURL='https://vikingnews.delmar.edu/osisoft-gifts-over-1-million-in-software-to-process-technology-program-and-enhances-training-for-industry-workforce', name='Pilot_Plant', namespace='hub_lab_data', status='onboarding'), 'Element': Node('Element', af_template='', asset_db='pilot.plant', description='', id='pilot.plant:F1EmcFWpYyj-GkWKwoVLgdSfpw7j7_VF8k6xGpdwANOjT1RQUElBRi1BQ0FEXFBJTE9UUExBTlRcUExBTlRcUElMT1QgUExBTlQ', name='Pilot Plant')}
<
>> Current anchor: (:Element {af_template: '', asset_db: 'pilot.plant', description: 'HOT-1 unit', id

@@@@@ Attribute {'name': 'E-102 Tube Side Inlet Temperature', 'id': 'pilot.plant:F1AbEcFWpYyj-GkWKwoVLgdSfpwEj__VF8k6xGpdwANOjT1RQVoJRJrL3JU68xcDh-QkhHgUElBRi1BQ0FEXFBJTE9UUExBTlRcUExBTlRcUElMT1QgUExBTlRcSE9ULTFcRS0xMDJ8RS0xMDIgVFVCRSBTSURFIElOTEVUIFRFTVBFUkFUVVJF', 'af_template': '', 'asset_db': 'pilot.plant', 'value': 'delmar.DELMAR_UNIT1_TI-101.PV', 'type': 'Single', 'is_stream': True, 'virtual': False, 'parent': 'E-102'}
@@@@@ PIPoint {'name': 'E-102 Tube Side Inlet Temperature', 'id': 'F1DPpjXwPKxU8EWdWQR_TqpF6QjC8AAAQUNBRC1QSURBLVZNMFxERUxNQVIuREVMTUFSX1VOSVQxX1RJLTEwMS5QVg', 'af_template': '', 'asset_db': 'pilot.plant', 'asset_id': 'E-102', 'column_name': 'E-102 Tube Side Inlet Temperature', 'stream_name': 'delmar.DELMAR_UNIT1_TI-101.PV', 'type': 'Single', 'uom': 'degree Fahrenheit', 'categories': ['Process Data'], 'description': '', 'pointsource': 'PIC', 'stream_id': 'PI_acad-pida-vm0_12172'}
@@@@@ Attribute {'name': 'E-102 Tube Side Outlet Pressure', 'id': 'pilot.plant:F1AbEcF

@@@@@ Attribute {'name': 'H-101 Outlet Temperature Controller Output', 'id': 'pilot.plant:F1AbEcFWpYyj-GkWKwoVLgdSfpwDD__VF8k6xGpdwANOjT1RQa3bo8P3EHEKOX_RoNhrOLgUElBRi1BQ0FEXFBJTE9UUExBTlRcUExBTlRcUElMT1QgUExBTlRcSE9ULTFcSFRSLTEwMXxILTEwMSBPVVRMRVQgVEVNUEVSQVRVUkUgQ09OVFJPTExFUiBPVVRQVVQ', 'af_template': '', 'asset_db': 'pilot.plant', 'value': 'delmar.DELMAR_UNIT1_TIC-104.OUT', 'type': 'Single', 'is_stream': True, 'virtual': False, 'parent': 'HTR-101'}
@@@@@ PIPoint {'name': 'H-101 Outlet Temperature Controller Output', 'id': 'F1DPpjXwPKxU8EWdWQR_TqpF6QiS8AAAQUNBRC1QSURBLVZNMFxERUxNQVIuREVMTUFSX1VOSVQxX1RJQy0xMDQuT1VU', 'af_template': '', 'asset_db': 'pilot.plant', 'asset_id': 'HTR-101', 'column_name': 'H-101 Outlet Temperature Controller Output', 'stream_name': 'delmar.DELMAR_UNIT1_TIC-104.OUT', 'type': 'Single', 'uom': 'percent', 'categories': ['Process Data'], 'description': '', 'pointsource': 'PIC', 'stream_id': 'PI_acad-pida-vm0_12169'}
@@@@@ Attribute {'name': 'H-101 Outlet Tempe

@@@@@ PIPoint {'name': 'Unit 1 Common Trouble Alarm', 'id': 'F1DPpjXwPKxU8EWdWQR_TqpF6QzC8AAAQUNBRC1QSURBLVZNMFxERUxNQVIuREVMTUFSX1VOSVQxX1hBLTEwMA', 'af_template': '', 'asset_db': 'pilot.plant', 'asset_id': 'TK-101', 'column_name': 'Unit 1 Common Trouble Alarm', 'stream_name': 'delmar.DELMAR_UNIT1_XA-100', 'type': 'String', 'uom': '', 'categories': [], 'description': '', 'pointsource': 'PIC', 'stream_id': 'PI_acad-pida-vm0_12236', 'digital_set_name': 'delmar.AlarmStatus'}
@@@@@ Element {'name': 'TK-101', 'id': 'pilot.plant:F1EmcFWpYyj-GkWKwoVLgdSfpwGD__VF8k6xGpdwANOjT1RQUElBRi1BQ0FEXFBJTE9UUExBTlRcUExBTlRcUElMT1QgUExBTlRcSE9ULTFcVEstMTAx', 'af_template': '', 'asset_db': 'pilot.plant', 'description': 'Cold side storage tank'}
@@@@@ Attribute {'name': 'P-102 Discharge pressure', 'id': 'pilot.plant:F1AbEcFWpYyj-GkWKwoVLgdSfpwDz__VF8k6xGpdwANOjT1RQw3opgUNdOkK9GqiCh9E-PAUElBRi1BQ0FEXFBJTE9UUExBTlRcUExBTlRcUElMT1QgUExBTlRcSE9ULTFcVEstMTAyfFAtMTAyIERJU0NIQVJHRSBQUkVTU1VSRQ', 'af_template': '

<<<<<< {'MergeElementHas_attribute': {'from': {'name': 'E-102'}, 'to': {'name': 'E-102 Tube Side Inlet Pressure'}}}
&&& Attribute from: {id: "pilot.plant:F1AbEcFWpYyj-GkWKwoVLgdSfpwEj__VF8k6xGpdwANOjT1RQ1XjYgbKDQkSzRJcWd8FbigUElBRi1BQ0FEXFBJTE9UUExBTlRcUExBTlRcUElMT1QgUExBTlRcSE9ULTFcRS0xMDJ8RS0xMDIgVFVCRSBTSURFIElOTEVUIFBSRVNTVVJF"}, to: {id: "F1DPpjXwPKxU8EWdWQR_TqpF6Qty8AAAQUNBRC1QSURBLVZNMFxERUxNQVIuREVMTUFSX1VOSVQxX1BJLTEwNS5QVg"}
<<<<<< {'MergeAttributeHas_dynamic': {'from': {'name': 'E-102 Tube Side Inlet Pressure'}, 'to': {'name': 'E-102 Tube Side Inlet Pressure'}}}
&&& Element from: {id: "pilot.plant:F1EmcFWpYyj-GkWKwoVLgdSfpwEj__VF8k6xGpdwANOjT1RQUElBRi1BQ0FEXFBJTE9UUExBTlRcUExBTlRcUElMT1QgUExBTlRcSE9ULTFcRS0xMDI"}, to: {id: "pilot.plant:F1AbEcFWpYyj-GkWKwoVLgdSfpwEj__VF8k6xGpdwANOjT1RQVoJRJrL3JU68xcDh-QkhHgUElBRi1BQ0FEXFBJTE9UUExBTlRcUExBTlRcUElMT1QgUExBTlRcSE9ULTFcRS0xMDJ8RS0xMDIgVFVCRSBTSURFIElOTEVUIFRFTVBFUkFUVVJF"}
<<<<<< {'MergeElementHas_attribute': {'from': {'name': '

...

<<<<<< {'MergeElementHas_attribute': {'from': {'name': 'Overhead Product'}, 'to': {'name': 'Bottoms Level'}}}
&&& Attribute from: {id: "pilot.plant:F1AbEcFWpYyj-GkWKwoVLgdSfpw9z7_VF8k6xGpdwANOjT1RQT5HhCZnCmUqjpO5ZTg4DEgUElBRi1BQ0FEXFBJTE9UUExBTlRcUExBTlRcUElMT1QgUExBTlRcSE9ULTNcT1ZFUkhFQUQgUFJPRFVDVHxCT1RUT01TIExFVkVM"}, to: {id: "F1DPpjXwPKxU8EWdWQR_TqpF6Qni8AAAQUNBRC1QSURBLVZNMFxERUxNQVIuREVMTUFSX1VOSVQzX0xJLTIxMC5QVg"}
<<<<<< {'MergeAttributeHas_dynamic': {'from': {'name': 'Bottoms Level'}, 'to': {'name': 'Bottoms Level'}}}
&&& Element from: {id: "pilot.plant:F1EmcFWpYyj-GkWKwoVLgdSfpw9z7_VF8k6xGpdwANOjT1RQUElBRi1BQ0FEXFBJTE9UUExBTlRcUExBTlRcUElMT1QgUExBTlRcSE9ULTNcT1ZFUkhFQUQgUFJPRFVDVA"}, to: {id: "pilot.plant:F1AbEcFWpYyj-GkWKwoVLgdSfpw9z7_VF8k6xGpdwANOjT1RQmo0WreIxiECHTEr3L3G0pwUElBRi1BQ0FEXFBJTE9UUExBTlRcUExBTlRcUElMT1QgUExBTlRcSE9ULTNcT1ZFUkhFQUQgUFJPRFVDVHxPVkVSSEVBRCBSRUNFSVZFUiBMRVZFTCBDT05UUk9MTEVSIE1PREU"}
<<<<<< {'MergeElementHas_attribute': {'from': {'name': 'Overhead P

<<<<<< {'MergeAttributeHas_dynamic': {'from': {'name': 'Reflux to Vac Tower Flow Controller PV'}, 'to': {'name': 'Reflux to Vac Tower Flow Controller PV'}}}
&&& Element from: {id: "pilot.plant:F1EmcFWpYyj-GkWKwoVLgdSfpw9z7_VF8k6xGpdwANOjT1RQUElBRi1BQ0FEXFBJTE9UUExBTlRcUExBTlRcUElMT1QgUExBTlRcSE9ULTNcT1ZFUkhFQUQgUFJPRFVDVA"}, to: {id: "pilot.plant:F1AbEcFWpYyj-GkWKwoVLgdSfpw9z7_VF8k6xGpdwANOjT1RQOT5IgTQlt0KHDLgTLuwKdAUElBRi1BQ0FEXFBJTE9UUExBTlRcUExBTlRcUElMT1QgUExBTlRcSE9ULTNcT1ZFUkhFQUQgUFJPRFVDVHxSRUZMVVggVE8gVkFDIFRPV0VSIEZMT1cgQ09OVFJPTExFUiBTRVRQT0lOVA"}
<<<<<< {'MergeElementHas_attribute': {'from': {'name': 'Overhead Product'}, 'to': {'name': 'Reflux to Vac Tower Flow Controller Setpoint'}}}
&&& Attribute from: {id: "pilot.plant:F1AbEcFWpYyj-GkWKwoVLgdSfpw9z7_VF8k6xGpdwANOjT1RQOT5IgTQlt0KHDLgTLuwKdAUElBRi1BQ0FEXFBJTE9UUExBTlRcUExBTlRcUElMT1QgUExBTlRcSE9ULTNcT1ZFUkhFQUQgUFJPRFVDVHxSRUZMVVggVE8gVkFDIFRPV0VSIEZMT1cgQ09OVFJPTExFUiBTRVRQT0lOVA"}, to: {id: "F1DPpjXwPKxU8EWdWQR_TqpF6QlS8

<<<<<< {'MergeAttributeHas_dynamic': {'from': {'name': 'V-240 Receiver Vacuum Controller PV'}, 'to': {'name': 'V-240 Receiver Vacuum Controller PV'}}}
&&& Element from: {id: "pilot.plant:F1EmcFWpYyj-GkWKwoVLgdSfpw9D7_VF8k6xGpdwANOjT1RQUElBRi1BQ0FEXFBJTE9UUExBTlRcUExBTlRcUElMT1QgUExBTlRcSE9ULTNcUFJFU1NVUkU"}, to: {id: "pilot.plant:F1AbEcFWpYyj-GkWKwoVLgdSfpw9D7_VF8k6xGpdwANOjT1RQ8qJBrXEv8EGdBSwvtAJY3QUElBRi1BQ0FEXFBJTE9UUExBTlRcUExBTlRcUElMT1QgUExBTlRcSE9ULTNcUFJFU1NVUkV8Vi0yNDAgUkVDRUlWRVIgVkFDVVVNIENPTlRST0xMRVIgU0VUUE9JTlQ"}
<<<<<< {'MergeElementHas_attribute': {'from': {'name': 'Pressure'}, 'to': {'name': 'V-240 Receiver Vacuum Controller Setpoint'}}}
&&& Attribute from: {id: "pilot.plant:F1AbEcFWpYyj-GkWKwoVLgdSfpw9D7_VF8k6xGpdwANOjT1RQ8qJBrXEv8EGdBSwvtAJY3QUElBRi1BQ0FEXFBJTE9UUExBTlRcUExBTlRcUElMT1QgUExBTlRcSE9ULTNcUFJFU1NVUkV8Vi0yNDAgUkVDRUlWRVIgVkFDVVVNIENPTlRST0xMRVIgU0VUUE9JTlQ"}, to: {id: "F1DPpjXwPKxU8EWdWQR_TqpF6Qly8AAAQUNBRC1QSURBLVZNMFxERUxNQVIuREVMTUFSX1VOSVQzX1BJQy0yNTA

<<<<<< {'MergeAttributeHas_dynamic': {'from': {'name': 'Overhead Pressure'}, 'to': {'name': 'Overhead Pressure'}}}
&&& Element from: {id: "pilot.plant:F1EmcFWpYyj-GkWKwoVLgdSfpw-j7_VF8k6xGpdwANOjT1RQUElBRi1BQ0FEXFBJTE9UUExBTlRcUExBTlRcUElMT1QgUExBTlRcSE9ULTNcVC0yMDA"}, to: {id: "pilot.plant:F1AbEcFWpYyj-GkWKwoVLgdSfpw-j7_VF8k6xGpdwANOjT1RQUUwR6MI5YE2pidg4aMm2DQUElBRi1BQ0FEXFBJTE9UUExBTlRcUExBTlRcUElMT1QgUExBTlRcSE9ULTNcVC0yMDB8T1ZFUkhFQUQgVE8gRkVFRCBGTE9X"}
<<<<<< {'MergeElementHas_attribute': {'from': {'name': 'T-200'}, 'to': {'name': 'Overhead to Feed Flow'}}}
&&& Attribute from: {id: "pilot.plant:F1AbEcFWpYyj-GkWKwoVLgdSfpw-j7_VF8k6xGpdwANOjT1RQUUwR6MI5YE2pidg4aMm2DQUElBRi1BQ0FEXFBJTE9UUExBTlRcUExBTlRcUElMT1QgUExBTlRcSE9ULTNcVC0yMDB8T1ZFUkhFQUQgVE8gRkVFRCBGTE9X"}, to: {id: "F1DPpjXwPKxU8EWdWQR_TqpF6QpS8AAAQUNBRC1QSURBLVZNMFxERUxNQVIuREVMTUFSX1VOSVQzX0ZJLTIxMC5QVg"}
<<<<<< {'MergeAttributeHas_dynamic': {'from': {'name': 'Overhead to Feed Flow'}, 'to': {'name': 'Overhead to Feed Flow'

<<<<<< {'MergeAttributeHas_dynamic': {'from': {'name': 'Reflux to VacTower Flow Controller Mode'}, 'to': {'name': 'Reflux to VacTower Flow Controller Mode'}}}
&&& Element from: {id: "pilot.plant:F1EmcFWpYyj-GkWKwoVLgdSfpw-j7_VF8k6xGpdwANOjT1RQUElBRi1BQ0FEXFBJTE9UUExBTlRcUExBTlRcUElMT1QgUExBTlRcSE9ULTNcVC0yMDA"}, to: {id: "pilot.plant:F1AbEcFWpYyj-GkWKwoVLgdSfpw-j7_VF8k6xGpdwANOjT1RQCOuAeXvmZ0SFozd4p-_FJwUElBRi1BQ0FEXFBJTE9UUExBTlRcUExBTlRcUElMT1QgUExBTlRcSE9ULTNcVC0yMDB8U1RFQU0gVE8gUkVCT0lMRVIgRkxPVyBDT05UUk9MTEVSIE1PREU"}
<<<<<< {'MergeElementHas_attribute': {'from': {'name': 'T-200'}, 'to': {'name': 'Steam to Reboiler Flow Controller Mode'}}}
&&& Attribute from: {id: "pilot.plant:F1AbEcFWpYyj-GkWKwoVLgdSfpw-j7_VF8k6xGpdwANOjT1RQCOuAeXvmZ0SFozd4p-_FJwUElBRi1BQ0FEXFBJTE9UUExBTlRcUExBTlRcUElMT1QgUExBTlRcSE9ULTNcVC0yMDB8U1RFQU0gVE8gUkVCT0lMRVIgRkxPVyBDT05UUk9MTEVSIE1PREU"}, to: {id: "F1DPpjXwPKxU8EWdWQR_TqpF6Qvi8AAAQUNBRC1QSURBLVZNMFxERUxNQVIuREVMTUFSX1VOSVQzX0ZJQy0yNzUuTU9ERQ"}
<<<<<< {

In [15]:
dataserver_urls = (requests.get(base_url + "/dataservers", auth=auth).json())["Items"]
for ds in dataserver_urls:
    if dataserver == ds["Name"]:
        print(ds)
        enums_url = ds["Links"]["EnumerationSets"]
        break
print(enums_url)
enums = requests.get(enums_url, auth=auth)
enums

{'WebId': 'F1DSpjXwPKxU8EWdWQR_TqpF6QQUNBRC1QSURBLVZNMA', 'Id': '3cf035a6-54ac-45f0-9d59-047f4eaa45e9', 'Name': 'acad-pida-vm0', 'Path': '\\\\PIServers[acad-pida-vm0]', 'IsConnected': True, 'ServerVersion': '3.4.405.1198', 'Links': {'Self': 'https://academicpi.osisoft.com/piwebapi/dataservers/F1DSpjXwPKxU8EWdWQR_TqpF6QQUNBRC1QSURBLVZNMA', 'Points': 'https://academicpi.osisoft.com/piwebapi/dataservers/F1DSpjXwPKxU8EWdWQR_TqpF6QQUNBRC1QSURBLVZNMA/points', 'EnumerationSets': 'https://academicpi.osisoft.com/piwebapi/dataservers/F1DSpjXwPKxU8EWdWQR_TqpF6QQUNBRC1QSURBLVZNMA/enumerationsets'}}
https://academicpi.osisoft.com/piwebapi/dataservers/F1DSpjXwPKxU8EWdWQR_TqpF6QQUNBRC1QSURBLVZNMA/enumerationsets


<Response [200]>

In [16]:
# Find all Digital Sets referred to by dataset PIPoint
query_pipoint = gql(
    """
query PIPoint($asset_db: String!) { 
    PIPoint(asset_db: $asset_db) {
        digital_set_name
        id
    }
}
"""
)

result = client.execute(
    query_pipoint, variable_values={"asset_db": ocs_asset_db}
)  # ocs_asset_db})
print(result)

digital_sets = [
    i["digital_set_name"]
    for i in result["PIPoint"]
    if i["digital_set_name"] is not None
]
# print(set(digital_sets))

dataset_enums = list(set(digital_sets))
dataset_enums

{'PIPoint': [{'digital_set_name': None, 'id': 'F1DPpjXwPKxU8EWdWQR_TqpF6Qmy8AAAQUNBRC1QSURBLVZNMFxERUxNQVIuREVMTUFSX1VOSVQxX1BJLTExMy5QVg'}, {'digital_set_name': None, 'id': 'F1DPpjXwPKxU8EWdWQR_TqpF6Q1i8AAAQUNBRC1QSURBLVZNMFxERUxNQVIuREVMTUFSX1VOSVQxX1RJLTEwNy5QVg'}, {'digital_set_name': None, 'id': 'F1DPpjXwPKxU8EWdWQR_TqpF6Q0y8AAAQUNBRC1QSURBLVZNMFxERUxNQVIuREVMTUFSX1VOSVQxX1BJLTExMC5QVg'}, {'digital_set_name': None, 'id': 'F1DPpjXwPKxU8EWdWQR_TqpF6Q2C8AAAQUNBRC1QSURBLVZNMFxERUxNQVIuREVMTUFSX1VOSVQxX1RJLTEwNi5QVg'}, {'digital_set_name': None, 'id': 'F1DPpjXwPKxU8EWdWQR_TqpF6Qty8AAAQUNBRC1QSURBLVZNMFxERUxNQVIuREVMTUFSX1VOSVQxX1BJLTEwNS5QVg'}, {'digital_set_name': None, 'id': 'F1DPpjXwPKxU8EWdWQR_TqpF6Q2i8AAAQUNBRC1QSURBLVZNMFxERUxNQVIuREVMTUFSX1VOSVQxX1RJLTEwNS5QVg'}, {'digital_set_name': None, 'id': 'F1DPpjXwPKxU8EWdWQR_TqpF6QjC8AAAQUNBRC1QSURBLVZNMFxERUxNQVIuREVMTUFSX1VOSVQxX1RJLTEwMS5QVg'}, {'digital_set_name': None, 'id': 'F1DPpjXwPKxU8EWdWQR_TqpF6QqC8AAAQUNBRC1QSURBLVZNMFxERUxNQ

['delmar.ControllerMode', 'delmar.AlarmStatus', 'delmar.RunStatus']

In [17]:
# Build DigitalState node with all states, add relationship :STATE_FROM from node with values in that DigitalState set
for digital_set in dataset_enums:
    # print("digital_set:", digital_set)
    d = {}
    for i in (enums.json())["Items"]:
        if i["Name"] == digital_set:
            print(f"==> processing {i}...")
            vals = requests.get(i["Links"]["Values"], auth=auth).json()
            for v in vals["Items"]:
                if v["Name"] != "undefined":
                    # print(f"{v['Value']} - {v['Name']}")
                    d[str(v["Value"])] = v["Name"]
            node = create_or_update_node(
                "DigitalState",
                digital_set,
                map_webid(i["WebId"]),
                asset_db=ocs_asset_db,
            )
            node.update(states=str(d))
            lgraph = [node]
            commit_graph(lgraph)
            digital_query = gql(
                """
                query PIPoint($asset_db: String, $digital_set: String) {
                    PIPoint(asset_db: $asset_db, digital_set_name: $digital_set) {
                        id
                    }
                }
            """
            )
            result = client.execute(
                digital_query,
                variable_values={
                    "asset_db": ocs_asset_db,
                    "digital_set": digital_set,
                },  # ocs_asset_db
            )
            print(result)
            digital_mutation = gql(
                """
               mutation Digital($from: _PIPointInput!, $to: _DigitalStateInput!) {
                   MergePIPointState_from(from: $from, to: $to) {
                       from {
                           name
                       }
                       to {
                           name
                       }
                   }
               }
               
            """
            )
            for p in result["PIPoint"]:
                result = client.execute(
                    digital_mutation,
                    variable_values={"from": {"id": p["id"]}, "to": {"id": node["id"]}},
                )
                print(">>>>", result)
            """
            points = [
                i["p"]
                for i in graph.run(
                    f"MATCH (p:PIPoint) WHERE p.digital_set_name = '{digital_set}' RETURN p"
                ).data()
            ]
            for p in points:
                rel = Relationship(p, "STATE_FROM", node)
                lgraph.extend([rel])

            break
            """

==> processing {'WebId': 'F1MSDpjXwPKxU8EWdWQR_TqpF6Q6UWqTn8EWZ0AMQAAADQAAAUElTRVJWRVJTW0FDQUQtUElEQS1WTTBdXFNUQVRFU0VUU1tERUxNQVIuQ09OVFJPTExFUk1PREVd', 'Id': '4eaa45e9-047f-9d59-0031-000000340000', 'Name': 'delmar.ControllerMode', 'Description': '', 'Path': '\\\\PIServers[acad-pida-vm0]\\StateSets[delmar.ControllerMode]', 'Links': {'Self': 'https://academicpi.osisoft.com/piwebapi/enumerationsets/F1MSDpjXwPKxU8EWdWQR_TqpF6Q6UWqTn8EWZ0AMQAAADQAAAUElTRVJWRVJTW0FDQUQtUElEQS1WTTBdXFNUQVRFU0VUU1tERUxNQVIuQ09OVFJPTExFUk1PREVd', 'DataServer': 'https://academicpi.osisoft.com/piwebapi/dataservers/F1DSpjXwPKxU8EWdWQR_TqpF6QQUNBRC1QSURBLVZNMA', 'Values': 'https://academicpi.osisoft.com/piwebapi/enumerationsets/F1MSDpjXwPKxU8EWdWQR_TqpF6Q6UWqTn8EWZ0AMQAAADQAAAUElTRVJWRVJTW0FDQUQtUElEQS1WTTBdXFNUQVRFU0VUU1tERUxNQVIuQ09OVFJPTExFUk1PREVd/enumerationvalues', 'Security': 'https://academicpi.osisoft.com/piwebapi/enumerationsets/F1MSDpjXwPKxU8EWdWQR_TqpF6Q6UWqTn8EWZ0AMQAAADQAAAUElTRVJWRVJTW0FDQUQtUElEQS


>> Graph root=(:DigitalState {af_template: '', asset_db: 'pilot.plant', id: 'pilot.plant:F1MSDpjXwPKxU8EWdWQR_TqpF6Q6UWqTn8EWZ0AMQAAADMAAAUElTRVJWRVJTW0FDQUQtUElEQS1WTTBdXFNUQVRFU0VUU1tERUxNQVIuUlVOU1RBVFVTXQ', name: 'delmar.RunStatus', states: "{'0': 'Not Running', '1': 'Running'}"})
@@@@@ DigitalState {'name': 'delmar.RunStatus', 'id': 'pilot.plant:F1MSDpjXwPKxU8EWdWQR_TqpF6Q6UWqTn8EWZ0AMQAAADMAAAUElTRVJWRVJTW0FDQUQtUElEQS1WTTBdXFNUQVRFU0VUU1tERUxNQVIuUlVOU1RBVFVTXQ', 'af_template': '', 'asset_db': 'pilot.plant', 'states': "{'0': 'Not Running', '1': 'Running'}"}
>> commit done in 0.01 secs
{'PIPoint': [{'id': 'F1DPpjXwPKxU8EWdWQR_TqpF6QxC8AAAQUNBRC1QSURBLVZNMFxERUxNQVIuREVMTUFSX1VOSVQxX1hCLTEwMUE'}, {'id': 'F1DPpjXwPKxU8EWdWQR_TqpF6QxS8AAAQUNBRC1QSURBLVZNMFxERUxNQVIuREVMTUFSX1VOSVQxX1hCLTEwMUI'}, {'id': 'F1DPpjXwPKxU8EWdWQR_TqpF6QuC8AAAQUNBRC1QSURBLVZNMFxERUxNQVIuREVMTUFSX1VOSVQxX1hCLTEwMg'}, {'id': 'F1DPpjXwPKxU8EWdWQR_TqpF6Qqy8AAAQUNBRC1QSURBLVZNMFxERUxNQVIuREVMTUFSX1VOSVQxX1hCLTE